In [1]:
import json
from collections import deque
from my_cube_solver import Cube
import copy
import numpy as np

In [2]:
def generate_move_table():
    """
    Generates a table mapping each move to the positions it affects and their new positions.

    Returns:
        dict: A dictionary where keys are move names and values are dictionaries
              mapping initial positions to final positions after the move.
    """
    cube = Cube()  # Create a fresh cube in solved state
    movement_table = {}

    # Positions that are not the center and thus their pieces' positions are tracked
    tracked_positions = []
    for i in range(3):
        for j in range(3):
            for k in range(3):
                if (i, j, k) != (1, 1, 1):  # Exclude the center position
                    tracked_positions.append((i, j, k))

    # For each possible move
    for move in cube.move_map.keys():
        # Get fresh cube for each move calculation
        test_cube = Cube()

        # Record initial piece IDs at each tracked position
        initial_piece_ids = {}
        for pos in tracked_positions:
            initial_piece_ids[pos] = test_cube._get_piece_at_position(pos)

        # Apply the move
        test_cube.apply_moves(move)

        # Record final positions based on moved piece IDs
        movements = {}
        for initial_pos in tracked_positions:
            piece_id_to_track = initial_piece_ids[initial_pos]
            final_pos = test_cube._get_position_of_piece(piece_id_to_track) # Find where that piece ended up
            if initial_pos != final_pos:
                movements[initial_pos] = final_pos

        # Store in table
        movement_table[move] = movements

    # Save to file
    with open('position_movement_table.json', 'w') as f: # Changed filename
        # Convert tuple positions to strings for JSON serialization
        serializable_table = {}
        for move, position_movements in movement_table.items():
            serializable_movements = {}
            for from_pos, to_pos in position_movements.items():
                from_pos_str = ','.join(map(str, from_pos))
                to_pos_str = ','.join(map(str, to_pos))
                serializable_movements[from_pos_str] = to_pos_str
            serializable_table[move] = serializable_movements

        json.dump(serializable_table, f, indent=2)

In [3]:
def build_single_piece_graph(all_moves, piece_type="edge"):
    """
    Precompute how each face turn affects a single piece's position
    (i, j, k). We do this for the solved cube, placing our 'test piece'
    in each valid position (corner or edge) and seeing where it moves.
    """
    # We only care about valid positions for the piece type
    valid_positions = []
    solved = Cube()
    edge_positions = solved.edge_positions
    corner_positions = solved.corner_positions
    
    if piece_type == 'edge':
        valid_positions = edge_positions
    else:
        valid_positions = corner_positions

    # Create a dictionary {position: {move: new_position}}
    # Then BFS will quickly find how many moves from start->end ignoring other pieces
    graph = {}

    for pos in valid_positions:
        graph[pos] = {}
        # Place a "test piece" ID (e.g., 999) at pos in a copy of a solved cube
        # so we can track that "test piece" alone.
        test_cube = copy.deepcopy(solved)
        test_cube.piece_current_positions[pos] = 999  # place test piece at 'pos'
        original_id = solved.piece_current_positions[pos]

        # Temporarily remove the original piece ID so it doesn't conflict
        solved_index = np.argwhere(test_cube.piece_current_positions == original_id)
        for (i_s, j_s, k_s) in solved_index:
            test_cube.piece_current_positions[i_s, j_s, k_s] = -1  # placeholder

        # For each move, see where 999 goes
        for move in all_moves:
            temp_cube = copy.deepcopy(test_cube)
            temp_cube.move_map[move]()

            # Find the new position of 999
            new_pos = None
            for i_ in range(3):
                for j_ in range(3):
                    for k_ in range(3):
                        if temp_cube.piece_current_positions[i_, j_, k_] == 999:
                            new_pos = (i_, j_, k_)
                            break
                    if new_pos is not None:
                        break

            # Store the transition
            graph[pos][move] = new_pos

        # Restore original piece ID in the solved cube copy
        test_cube.piece_current_positions[pos] = original_id

    return graph

def calculate_distance_table_ignoring_others(piece_type, filename:str): # Removed piece_ids arg, now using valid positions directly
    """
    Builds a position graph for the piece type, then BFSes in that position graph
    from the start position to the target position. This ignores the rest of the puzzle.
    The distance table is keyed by positions, not piece IDs.
    """
    solved_cube = Cube()
    all_moves = list(solved_cube.move_map.keys())
    core_moves = copy.deepcopy(all_moves)
    for move in ['M', 'E', 'S', 'm', 'e', 's']:
        core_moves.remove(move)
    pos_graph = build_single_piece_graph(core_moves, piece_type=piece_type) # Use core_moves here

    valid_positions = list(pos_graph.keys()) # Get valid positions directly from graph keys

    distance_table = {}
    for start_pos in valid_positions: # Iterate over positions, not piece_ids
        for target_pos in valid_positions:
            if start_pos == target_pos:
                distance_table[(start_pos, target_pos)] = 0
                continue

            if (target_pos, start_pos) in distance_table: #symmetry optimization
                distance_table[(start_pos, target_pos)] = distance_table[(target_pos, start_pos)]
                continue

            # BFS in the position graph from start_pos to target_pos
            visited = set([start_pos])
            queue = deque([(start_pos, 0)])
            found_distance = -1

            while queue:
                current_pos, dist = queue.popleft()
                if current_pos == target_pos:
                    found_distance = dist
                    break
                # Explore all possible moves from current_pos in the graph
                for move in pos_graph[current_pos]:
                    next_pos = pos_graph[current_pos][move]
                    if next_pos not in visited:
                        visited.add(next_pos)
                        queue.append((next_pos, dist + 1))

            distance_table[(start_pos, target_pos)] = found_distance

    serializable_table = {}
    for pos_pair, dist in distance_table.items(): # Key is now position pair
        serializable_table[str(pos_pair)] = dist
    with open(filename, 'w') as f:
        json.dump(serializable_table, f, indent=2)
    

In [4]:
generate_move_table()
calculate_distance_table_ignoring_others("edge", "edge_position_distance_table.json")
calculate_distance_table_ignoring_others("corner", "corner_position_distance_table.json")

Successfully loaded tables: edge_distances, corner_distances, movements
Successfully loaded tables: edge_distances, corner_distances, movements
Successfully loaded tables: edge_distances, corner_distances, movements
Successfully loaded tables: edge_distances, corner_distances, movements
Successfully loaded tables: edge_distances, corner_distances, movements
Successfully loaded tables: edge_distances, corner_distances, movements
Successfully loaded tables: edge_distances, corner_distances, movements
Successfully loaded tables: edge_distances, corner_distances, movements
Successfully loaded tables: edge_distances, corner_distances, movements
Successfully loaded tables: edge_distances, corner_distances, movements
Successfully loaded tables: edge_distances, corner_distances, movements
Successfully loaded tables: edge_distances, corner_distances, movements
Successfully loaded tables: edge_distances, corner_distances, movements
Successfully loaded tables: edge_distances, corner_distances, mo